# Découverte de l’API XML *ElementTree*

L’API *ElementTree* analyse tout document XML comme un document structuré afin de le représenter sous forme d’arbre. Elle procure également des méthodes pour en manipuler les nœuds (création, modification, suppression).

Elle se charge en important le module `xml.etree.ElementTree` :

In [ ]:
import xml.etree.ElementTree as ET

## Analyser du code XML

Une méthode `.parse()` qui prend en argument le chemin vers un fichier XML permet d’en modéliser la structure. Généralement, la première opération qui suit consiste à récupérer l’élément racine du document afin de s’en servir comme point de référence :

In [ ]:
tree = ET.parse('./data/constellations.xml')
root = tree.getroot()

Comme tout élément XML, la racine dispose d’une étiquette ainsi que d’un dictionnaire pour recenser ses attributs. Et si l’élément accueille du contenu textuel, la propriété `text` permettrait de l’afficher.

In [ ]:
print(root.tag)
print(root.text)
print(root.attrib)

Un élément pouvant accueillir des éléments enfants avec leurs propres attributs, il est possible d’itérer dessus :

In [ ]:
for child in root:
    print(child.tag, child.attrib)

## Rechercher des éléments

L’API met à disposition deux méthodes pour rechercher des éléments à l’intérieur de la structure. La première, `.iter()`, parcourt tout l’arbre à la recherche de la correspondance.

In [ ]:
# from root, looking for every area tag
for area in root.iter('area'):
    print(area.text, area.attrib['unit'])

La seconde méthode, `.findall()`, s’arrête quant à elle aux enfants directs. Citons également la méthode `.find()` qui recherche la première correspondance d’un élément, et la méthode `.get()` qui permet d’accéder à un attribut spécifié en argument.

In [ ]:
# for each child of root
for child in root.findall('constellation'):
    name = child.find('name')
    name_fr = name.get('fr')
    print(f"{name.text} ({name_fr})")

## Créer des éléments et des sous-éléments

Les fonctions `Element()` et `SubElement()` figurent la meilleure façon de créer rapidement une arborescence XML. La première prend comme argument une étiquette ; la seconde prend comme premier argument l’élément auquel elle se rattache et en second argument une étiquette. Elles acceptent toutes deux un dictionnaire pour les attributs à définir.

Enfin, la méthode `.write()` permet de sérialiser l’arborescence dans un document XML.

In [ ]:
# open new file
with open('./data/planets.xml', 'wb') as xmlfile:

    # root element to tree structure
    root = ET.Element('planets')
    tree = ET.ElementTree(root)

    # sub elements, childs of root
    earth = ET.SubElement(root, 'planet', {'name': 'Earth'})
    mars = ET.SubElement(root, 'planet', {'name': 'Mars'})

    # sub elements under each 'planet'
    earth_surface = ET.SubElement(earth, 'surface', {'unit': 'km2'})
    mars_surface = ET.SubElement(mars, 'surface', {'unit': 'km2'})

    # textual content
    earth_surface.text = '510067420'
    mars_surface.text = '144798500'

    # serialize tree in file
    tree.write(xmlfile)

## Modifier des éléments

Les opérations essentielles de modification d’un élément sont assurées par les méthodes suivantes :
- `.set()` pour agir sur les attributs
- `.append()` pour ajouter un élément fils
- `.remove()` pour supprimer un élément fils

Pour remplacer le contenu textuel d’un élément, il suffit de remplacer le contenu de la propriété `text`. Et pour supprimer un attribut, utiliser l’instruction `del`.

In [ ]:
tree = ET.parse('./data/constellations.xml')
root = tree.getroot()

for idx, child in enumerate(root.findall('constellation')):

    # add an 'id' attribute
    child.set('id', str(idx + 1))

    # element '<name/>'
    name = child.find('name')

    # <name_fr/>
    name_fr = ET.Element('name_fr')
    name_fr.text = name.get('fr')
    child.append(name_fr)

    # attrib 'fr' useless
    del name.attrib['fr']

tree.write('./data/constellations_new.xml')

## Considérations sur l’indentation du code XML

Les espaces entre les éléments n’ont aucune utilité pour une machine, elle servent uniquement pour le confort visuel. Le code ci-dessous, correctement indenté, est parfaitement agréable à la vue :

```xml
<constellation origin="Lacaille">
    <name fr="La Carène">Carina</name>
    <areas>
        <area unit="deg2">494.184</area>
        <area unit="percent">1.20</area>
    </areas>
</constellation>
```

Pour une machine, il a en fait exactement la même signification que le code ci-dessous :

```xml
<constellation origin="Lacaille"><name fr="La Carène">Carina</name><areas><area unit="deg2">494.184</area><area unit="percent">1.20</area></areas></constellation>
```

Le module *ElementTree* ne se charge pas d’écrire un code parfaitement indenté. Deux solutions existent :
- soit rajouter manuellement des retours à la ligne (`\n`) et des tabulations (`\t`) au moment de l’écriture ;
- soit utiliser à la lecture un module qui réalise ce travail.

Le module `minidom` propose deux méthodes adéquates pour formater correctement un arbre XML. La première, `.parseString()` analyse une chaîne XML ; la seconde, `.parse()` prend comme argument un chemin vers un fichier. Pour voir le rendu, il faut invoquer la méthode `.toprettyxml()` :

In [ ]:
import xml.dom.minidom

xml_string = '<constellation origin="Lacaille"><name fr="La Carène">Carina</name><areas><area unit="deg2">494.184</area><area unit="percent">1.20</area></areas></constellation>'

dom = xml.dom.minidom.parseString(xml_string)
print(dom.toprettyxml())